- Thai-HomeworkGen-v4
- Qwen3-4B-Thinking-2507 (Base model)
- ChatGPT-5
- Gemini 2.5 Pro

In [12]:
import os
from os import environ
import pandas as pd
import requests
import json
from itertools import combinations, product
from joblib import Parallel, delayed
import time
from typing import List, Dict, Any, Tuple
import random
from tqdm.auto import tqdm

### Setting

In [26]:
MAX_NUM_PROBLEMS = 1
BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = [
    "multiple_choice",
    # "short_answer"
    ]
ADDITIONAL_REQUIREMENTS = "โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด"

# For parallel processing
N_JOBS = 4  # Adjust based on your system and API rate limits
BATCH_SIZE = N_JOBS * 2
DELAY_BETWEEN_REQUESTS = 1  # seconds

# ------------------------------------------------------------------------------

# Generate Bloom level combinations (1-2 levels max by default, can be increased)
BLOOM_COMBINATIONS = []
for r in range(1, 4):  # 1 to 3 bloom levels per question
    BLOOM_COMBINATIONS.extend(combinations(BLOOM_LEVELS, r))

# Convert combinations to list of lists for easier handling
BLOOM_COMBINATIONS = [list(combo) for combo in BLOOM_COMBINATIONS]

NUM_PROBLEMS = list(range(1, MAX_NUM_PROBLEMS + 1))

# Create all possible combinations of parameters
all_combinations = list(product(
    TOPICS,                   # topic
    GRADE_LEVELS,             # grade level
    QUESTION_TYPES,           # question type
    LEVELS,                   # difficulty
    BLOOM_COMBINATIONS,       # bloom levels
    NUM_PROBLEMS,             # num problems
))

print(f"Total possible combinations: {len(all_combinations)}")
print(f"Bloom combinations: {len(BLOOM_COMBINATIONS)}")
print(f"Sample combinations:")
for i, combo in enumerate(all_combinations[-5:]):
    topic, grade, q_type, difficulty, bloom, num_problems = combo
    print(f"  {i+1}. Topic: {topic}, Grade: {grade}, Type: {q_type}, Difficulty: {difficulty}, Bloom: {bloom}, Num Problems: {num_problems}")

Total possible combinations: 123
Bloom combinations: 41
Sample combinations:
  1. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ง่าย, Bloom: ['เข้าใจ', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 1
  2. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ง่าย, Bloom: ['นำไปใช้', 'วิเคราะห์', 'ประเมิน'], Num Problems: 1
  3. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ง่าย, Bloom: ['นำไปใช้', 'วิเคราะห์', 'สร้างสรรค์'], Num Problems: 1
  4. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ง่าย, Bloom: ['นำไปใช้', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 1
  5. Topic: พีชคณิต, Grade: ม.6, Type: multiple_choice, Difficulty: ง่าย, Bloom: ['วิเคราะห์', 'ประเมิน', 'สร้างสรรค์'], Num Problems: 1


In [48]:
# --- Generate all combinations for Thai-HomeworkGen-v4 ---
from itertools import combinations, product

BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = ["multiple_choice"]

# สร้าง bloom combinations: เดี่ยวและคู่ (ไม่ซ้ำ)
bloom_combinations = [[b] for b in BLOOM_LEVELS]
bloom_combinations += [list(pair) for pair in combinations(BLOOM_LEVELS, 2)]

# สร้าง combinations ทั้งหมด
all_combinations = list(product(
    TOPICS,
    GRADE_LEVELS,
    QUESTION_TYPES,
    LEVELS,
    bloom_combinations
))

print(f"จำนวนโจทย์ทั้งหมดที่ต้องสร้าง: {len(all_combinations)}")
print(f"ตัวอย่าง 5 ข้อแรก:")
for i, combo in enumerate(all_combinations[:5]):
    topic, grade, qtype, level, bloom = combo
    print(f"{i+1}. หัวข้อ: {topic}, ชั้น: {grade}, ประเภท: {qtype}, ความยาก: {level}, Bloom: {bloom}")

จำนวนโจทย์ทั้งหมดที่ต้องสร้าง: 189
ตัวอย่าง 5 ข้อแรก:
1. หัวข้อ: พีชคณิต, ชั้น: ม.4, ประเภท: multiple_choice, ความยาก: ง่าย, Bloom: ['จำ']
2. หัวข้อ: พีชคณิต, ชั้น: ม.4, ประเภท: multiple_choice, ความยาก: ง่าย, Bloom: ['เข้าใจ']
3. หัวข้อ: พีชคณิต, ชั้น: ม.4, ประเภท: multiple_choice, ความยาก: ง่าย, Bloom: ['นำไปใช้']
4. หัวข้อ: พีชคณิต, ชั้น: ม.4, ประเภท: multiple_choice, ความยาก: ง่าย, Bloom: ['วิเคราะห์']
5. หัวข้อ: พีชคณิต, ชั้น: ม.4, ประเภท: multiple_choice, ความยาก: ง่าย, Bloom: ['ประเมิน']


In [ ]:
def create_user_prompt(topic: str, grade_level: str, question_type: str, difficulty: str, bloom_levels: List[str], num_problems: int = 1, additional_requirements: str = ADDITIONAL_REQUIREMENTS) -> str:
    bloom_str = ", ".join(bloom_levels)
    prompt = f"""จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: {topic}
2. สำหรับนักเรียน: {grade_level}
3. รูปแบบ: {question_type}
4. ความยาก: {difficulty}
5. bloom level: {bloom_str}
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: {additional_requirements}"""

    return prompt

### Thai-HomeworkGen-v4

In [89]:
BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = ["multiple_choice"]

In [90]:
bloom_combinations = [[b] for b in BLOOM_LEVELS] + [list(pair) for pair in combinations(BLOOM_LEVELS, 2)]
all_combinations = list(product(TOPICS, GRADE_LEVELS, QUESTION_TYPES, LEVELS, bloom_combinations))
print(f"Total combinations: {len(all_combinations)}")

Total combinations: 189


In [91]:
def create_user_prompt(topic, grade_level, question_type, difficulty, bloom_levels, num_problems=1):
    bloom_str = ", ".join(bloom_levels)
    prompt = f"""จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: {topic}
2. สำหรับนักเรียน: {grade_level}
3. รูปแบบ: {question_type}
4. ความยาก: {difficulty}
5. bloom level: {bloom_str}
6. จำนวน: {num_problems} ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด"""
    return prompt

In [ ]:
RUNPOD_URL = "https://api.runpod.ai/v2/57uty6p5a5zfdt"
API_KEY = os.getenv("RUNPOD_API_KEY")
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

In [104]:
def call_runpod_api(user_prompt, system_prompt=SYSTEM_PROMPT):
    data = {
        "input": {
            "prompt": f"{system_prompt}\n\nUser: {user_prompt}\nAssistant:",
            "sampling_params": {"max_tokens": 9216}
        }
    }
    resp = requests.post(f"{RUNPOD_URL}/runsync", headers=HEADERS, json=data)
    if resp.status_code == 200:
        resp_json = resp.json()
        return resp_json.get("id")
    return None


def poll_runpod_result(run_id, poll_interval=2, timeout=180):
    url = f"{RUNPOD_URL}/status/{run_id}"
    start = time.time()
    while True:
        resp = requests.post(url, headers=HEADERS)
        if resp.status_code == 200:
            result = resp.json()
            status = result.get("status")
            if status == "COMPLETED":
                return result
            if status in ("FAILED", "CANCELLED"):
                return None
        if time.time() - start > timeout:
            return None
        time.sleep(poll_interval)

In [105]:
def extract_think_and_content(output_str):
    think, content = "", ""
    if "<think>" in output_str and "</think>" in output_str:
        think = output_str.split("<think>")[1].split("</think>")[0].replace("\n", " ").strip()
    # Keep everything from <questions> ... to the end (single block) and remove newlines
    if "<questions>" in output_str:
        content = output_str.split("<questions>", 1)[1]
        content = "<questions>" + content
        content = content.replace("\n", " ").strip()
    return think, content


def save_result(think, content, run_id, params, result_meta=None, outdir="results"):
    """Save per-sample JSON file with think/content and metadata."""
    os.makedirs(outdir, exist_ok=True)
    record = {
        "id": run_id,
        "topic": params[0],
        "grade": params[1],
        "qtype": params[2],
        "level": params[3],
        "bloom": params[4],
        "think": think,
        "content": content,
        "meta": result_meta or {}
    }
    with open(f"{outdir}/result_{run_id}.json", "w", encoding="utf-8") as f:
        json.dump(record, f, ensure_ascii=False)

In [106]:
def _first_text_from_result(result: dict) -> str:
    """Try to extract the first text/token string from RunPod result."""
    try:
        out = result.get("output", [])
        if not out:
            return ""
        # Common layout: output[0]["choices"][0]["tokens"] -> list of strings
        choices = out[0].get("choices", [])
        if choices:
            msg = choices[0]
            # tokens may be an array of chunks: join
            tokens = msg.get("tokens")
            if isinstance(tokens, list):
                return "".join(tokens)
            # or message content
            content = msg.get("message", {}).get("content")
            if isinstance(content, str):
                return content
        # Fallback: stringify
        return json.dumps(out[0], ensure_ascii=False)
    except Exception:
        return ""


def process_one_combination(params, outdir="results"):
    topic, grade, qtype, level, bloom = params
    user_prompt = create_user_prompt(topic, grade, qtype, level, bloom)
    run_id = call_runpod_api(user_prompt)
    if not run_id:
        return None
    result = poll_runpod_result(run_id)
    if not result:
        return None
    output_str = _first_text_from_result(result)
    if not isinstance(output_str, str):
        output_str = str(output_str)
    think, content = extract_think_and_content(output_str)
    # Save as JSON per sample
    meta = {
        k: result.get(k) for k in ("delayTime", "executionTime", "status", "workerId") if k in result
    }
    save_result(think, content, run_id, params=params, result_meta=meta, outdir=outdir)
    return {
        "id": run_id,
        "topic": topic,
        "grade": grade,
        "qtype": qtype,
        "level": level,
        "bloom": bloom,
        "think": think,
        "content": content
    }

In [107]:
# ทดสอบระบบ (full system test 1 รอบ)
test_result = process_one_combination(all_combinations[0], outdir="test_results")
print("Test result:", test_result)

Test result: {'id': 'sync-a9606699-130f-446d-9dc9-22afbf6c1984-e1', 'topic': 'พีชคณิต', 'grade': 'ม.4', 'qtype': 'multiple_choice', 'level': 'ง่าย', 'bloom': ['จำ'], 'think': 'Alright, let\'s tackle this problem. The user wants me to create a high-quality math problem in algebra for 10th graders (ม.4), multiple-choice format, easy difficulty, Bloom\'s level "จำ" (Remember). It needs 4 options with 3 distractors based on common calculation errors.  First, I need to recall what Bloom\'s "Remember" level entails. It\'s about recalling facts, formulas, or basic procedures. So the question should test if students can remember a fundamental algebra concept or formula without applying it to a new scenario.  Since it\'s algebra for 10th grade, possible topics could be linear equations, quadratic formulas, exponent rules, etc. But since it\'s easy, maybe a simple equation or formula.  Let me think of a standard formula they should remember. For example, the slope-intercept form of a line: y = m

In [108]:
# Full system run (parallel) with tqdm progress
N_JOBS = 3
results = []
for res in tqdm(
    Parallel(n_jobs=N_JOBS, return_as="generator")(
        delayed(process_one_combination)(params) for params in all_combinations
    ),
    total=len(all_combinations),
    desc="Generating"
):
    results.append(res)

Generating: 100%|██████████| 189/189 [3:08:29<00:00, 59.84s/it]  


In [109]:
# Save
results = [r for r in results if r]
df = pd.DataFrame(results)
df.to_csv("all_results.csv", index=False, encoding="utf-8-sig")
df.to_parquet("all_results.parquet", index=False)

In [115]:
# Filter grade
len(df[df["grade"] == "ม.6"])

51

#### Default

In [ ]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer xxx'
}

full_prompt = f"{SYSTEM_PROMPT}\n\nUser: {USER_PROMPT}\nAssistant:"

data = {
    'input': {
        "prompt": full_prompt,
        "sampling_params": {
            "max_tokens": 9216
        }
    },
}

response = requests.get('https://api.runpod.ai/v2/57uty6p5a5zfdt/status/84e071b0-49b6-49d3-b6ba-4e56a7afab56-e2', headers=headers, json=data)
print(response.status_code)

200


In [76]:
response.content

b'{"delayTime":121181,"executionTime":262458,"id":"84e071b0-49b6-49d3-b6ba-4e56a7afab56-e2","output":[{"choices":[{"tokens":["<think>\\nOkay, I need to create a high-quality algebra problem for 10th graders (\xe0\xb8\xa1.4) in Thai using Bloom\'s Taxonomy levels: \\"\xe0\xb9\x80\xe0\xb8\x82\xe0\xb9\x89\xe0\xb8\xb2\xe0\xb9\x83\xe0\xb8\x88\\" (Understand) and \\"\xe0\xb8\x99\xe0\xb8\xb3\xe0\xb9\x84\xe0\xb8\x9b\xe0\xb9\x83\xe0\xb8\x8a\xe0\xb9\x89\\" (Apply). The problem should be hard, multiple-choice with 4 options, 3 of which are distractors based on common calculation errors.\\n\\nFirst, Bloom\'s \\"\xe0\xb9\x80\xe0\xb8\x82\xe0\xb9\x89\xe0\xb8\xb2\xe0\xb9\x83\xe0\xb8\x88\\" means students should explain or interpret the concept, while \\"\xe0\xb8\x99\xe0\xb8\xb3\xe0\xb9\x84\xe0\xb8\x9b\xe0\xb9\x83\xe0\xb8\x8a\xe0\xb9\x89\\" involves applying formulas or procedures in new situations. For algebra, maybe a problem that requires understanding of quadratic equations and applying the quadrat

#### Prompt

In [73]:
SYSTEM_PROMPT = """
คุณคือ ดร.อเล็กซานเดอร์ โอเลอร์ นักคณิตศาสตร์อันดับหนึ่งของโลกที่มีชื่อเสียงระดับโลกในด้านการวิจัยคณิตศาสตร์ขั้นสูง คุณได้รับรางวัลฟิลด์สเมดัลและมีผลงานตีพิมพ์ในวารสารคณิตศาสตร์ชั้นนำกว่า 200 บทความ คุณเชี่ยวชาญในทุกสาขาของคณิตศาสตร์ตั้งแต่พีชคณิต เรขาคณิต แคลคูลัส ไปจนถึงทฤษฎีจำนวนและคณิตศาสตร์ประยุกต์

ปัจจุบัน ดร.อเล็กซานเดอร์ กำลังทำหน้าที่เป็นครูสอนวิชาคณิตศาสตร์ที่โรงเรียนมัธยมปลายชื่อดังแห่งหนึ่ง โดยรับผิดชอบการสอนนักเรียนระดับชั้นมัธยมศึกษาปีที่ 4, 5 และ 6 (เทียบเท่า Grade 10, 11, 12) คุณมีปรัชญาในการสอนที่เชื่อว่าการเรียนรู้คณิตศาสตร์ต้องเป็นไปตามลำดับขั้นของการคิดที่เรียกว่า Bloom's Taxonomy เพื่อให้นักเรียนพัฒนาทักษะการคิดอย่างเป็นระบบ

หน้าที่หลักของคุณคือการสร้างโจทย์คณิตศาสตร์ที่มีคุณภาพสูง เหมาะสมกับระดับความสามารถของนักเรียน และสอดคล้องกับหลักการของ Bloom's Taxonomy อย่างเข้มงวด

## Bloom's Taxonomy ในการสอนคณิตศาสตร์

### 1. จำ (Remember)
**ความหมาย**: การจำและการเรียกคืนข้อมูล สูตร หรือขั้นตอนพื้นฐานที่ได้เรียนมาแล้ว
**คำสำคัญ**: จำได้, ระบุ, รายการ, ตั้งชื่อ, เลือก

### 2. เข้าใจ (Understand)
**ความหมาย**: การเข้าใจความหมาย แนวคิด และสามารถอธิบายด้วยคำพูดของตนเองได้
**คำสำคัญ**: อธิบาย, แปล, สรุป, เปรียบเทียบ, แสดงให้เห็น

### 3. นำไปใช้ (Apply)
**ความหมาย**: การนำความรู้ สูตร หรือขั้นตอนที่เรียนมาไปใช้แก้ปัญหาในสถานการณ์ใหม่
**คำสำคัญ**: คำนวณ, แก้, ใช้, แสดง, ดำเนินการ

### 4. วิเคราะห์ (Analyze)
**ความหมาย**: การแยกแยะองค์ประกอบ วิเคราะห์ความสัมพันธ์ และเข้าใจโครงสร้างของปัญหา
**คำสำคัญ**: แยกแยะ, เปรียบเทียบ, ตรวจสอบ, ทดสอบ, วิเคราะห์

### 5. ประเมิน (Evaluate)
**ความหมาย**: การตัดสิน ประเมินค่า ให้เหตุผล และแสดงความคิดเห็นโดยใช้เกณฑ์ที่กำหนด
**คำสำคัญ**: ตัดสิน, ประเมิน, วิจารณ์, แสดงความคิดเห็น, ให้เหตุผล

### 6. สร้างสรรค์ (Create)
**ความหมาย**: การสร้างสรรค์สิ่งใหม่ ออกแบบ วางแผน หรือสร้างโจทย์ปัญหาขึ้นมาเอง
**คำสำคัญ**: สร้าง, ออกแบบ, วางแผน, เสนอ, พัฒนา

## ตัวอย่างโจทย์ตาม Bloom's Taxonomy

### จำ (Remember)
โจทย์: สูตรหาพื้นที่วงกลมคือข้อใด
A) $ A = \pi r^2 $  B) $ A = 2\pi r $  C) $ A = \pi d $  D) $ A = \frac{1}{2}\pi r^2 $

### เข้าใจ (Understand)
โจทย์: อธิบายความหมายของ $ \frac{dy}{dx} = 3x^2 $ ในทางเรขาคณิต

### นำไปใช้ (Apply)
โจทย์: จงหาค่า $ x $ จากสมการ $ 2x + 5 = 13 $

### วิเคราะห์ (Analyze)
โจทย์: เปรียบเทียบพฤติกรรมของฟังก์ชัน $ f(x) = x^2 $ และ $ g(x) = x^3 $ เมื่อ $ x > 1 $

### ประเมิน (Evaluate)
โจทย์: นักเรียนคนหนึ่งอ้างว่า $ \sqrt{a + b} = \sqrt{a} + \sqrt{b} $ เสมอ จงประเมินว่าข้อความนี้ถูกหรือผิด พร้อมให้เหตุผล

### สร้างสรรค์ (Create)
โจทย์: ออกแบบฟังก์ชันกำลังสองที่มีจุดยอดอยู่ที่ $ (2, -3) $ และผ่านจุด $ (0, 1) $

## ตัวอย่างโจทย์แบบผสม 5 โจทย์

### โจทย์ที่ 1 (เข้าใจ + นำไปใช้)
หากฟังก์ชัน $ f(x) = 2x - 3 $ จงหาค่า $ f(5) $ และอธิบายความหมายของผลลัพธ์

### โจทย์ที่ 2 (วิเคราะห์ + ประเมิน)
เปรียบเทียบวิธีแก้สมการ $ x^2 - 5x + 6 = 0 $ ด้วยการแยกตัวประกอบและสูตรกำลังสอง แล้วประเมินว่าวิธีใดมีประสิทธิภาพมากกว่า

### โจทย์ที่ 3 (นำไปใช้ + วิเคราะห์)
ร้านค้าแห่งหนึ่งขายสินค้าในราคา $ 100x - x^2 $ บาท เมื่อขาย $ x $ ชิ้น จงหาจำนวนสินค้าที่ขายได้เงินสูงสุด

### โจทย์ที่ 4 (เข้าใจ + สร้างสรรค์)
จากกราฟ $ y = \sin x $ จงสร้างฟังก์ชันใหม่ที่มีแอมพลิจูดเป็น 3 เท่า และอธิบายการเปลี่ยนแปลง

### โจทย์ที่ 5 (วิเคราะห์ + ประเมิน + สร้างสรรค์)
นักเรียนสองคนแก้โจทย์หาค่าสูงสุดของ $ f(x) = -x^2 + 4x + 1 $ ได้คำตอบต่างกัน คนหนึ่งได้ $ x = 2 $ อีกคนได้ $ x = -2 $ จงวิเคราะห์ว่าใครถูก ประเมินข้อผิดพลาด และสร้างวิธีตรวจสอบคำตอบ

---

## คำสั่งการทำงาน

เมื่อได้รับโจทย์ให้สร้างข้อสอบคณิตศาสตร์ ให้ทำตามขั้นตอนดังนี้:

### ขั้นตอนที่ 1: การคิดและวางแผน
ในแท็ก `<think></think>` ให้ทำการ:
1. **คิดโจทย์เป็นภาษาอังกฤษก่อน** - เพื่อให้ได้แนวคิดที่ชัดเจน
2. **ทดลองหาคำตอบ** - ตรวจสอบว่าโจทย์สามารถแก้ได้จริง
3. **หากแก้ไม่ได้ ให้เปลี่ยนโจทย์** - แล้วทดลองใหม่จนได้โจทย์ที่ดี
4. **เมื่อได้โจทย์ที่สมบูรณ์แล้ว** - สร้างตัวเลือกคำตอบสำหรับ multiple choice (หากต้องการ)
5. **ตรวจสอบความสอดคล้องกับ Bloom's taxonomy** ที่กำหนด

### ขั้นตอนที่ 2: การเขียนผลลัพธ์
หลังจาก `<think></think>` แล้ว ให้เขียนโจทย์ในรูปแบบ XML ตามตัวอย่างนี้:

```xml
<questions>
  <question>
    <text>โจทย์คณิตศาสตร์ที่มี KaTeX formatting เช่น $ 4x + 3 = 2x + 9 $</text>
    <type>multiple_choice</type>
    <options>
      <option>$ ตัวเลือก1 $</option>
      <option>$ ตัวเลือก2 $</option>
      <option>$ ตัวเลือก3 $</option>
      <option>$ ตัวเลือก4 $</option>
    </options>
    <correct_answer>$ คำตอบที่ถูก $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: อธิบายด้วย KaTeX เช่น $ 4x - 2x + 3 = 9 $
      ขั้นตอนที่ 2: $ 2x + 3 = 9 $
      ขั้นตอนที่ 3: $ 2x = 6 $
      ขั้นตอนที่ 4: $ x = 3 $
    </explanation>
    <score>2</score>
    <difficulty>ง่าย</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
      <level>วิเคราะห์</level>
    </bloom_levels>
  </question>

  <question>
    <text>จงหาค่า $ x $ ที่ทำให้ $ 3x - 7 = 2x + 5 $</text>
    <type>short_answer</type>
    <correct_answer>$ x = 12 $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: นำ $ 2x $ ไปยังข้างซ้าย และ $ 7 $ ไปยังข้างขวา
      $ 3x - 2x = 5 + 7 $
      ขั้นตอนที่ 2: คำนวณ
      $ x = 12 $
    </explanation>
    <score>2</score>
    <difficulty>ปานกลาง</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
    </bloom_levels>
  </question>
</questions>
```

## หมายเหตุสำคัญ

### รูปแบบการเขียน:
- **ใช้ KaTeX format** `$ ... $` สำหรับตัวเลข สูตร และนิพจน์คณิตศาสตร์ทั้งหมด
- **ใช้คำไทยสำหรับ Bloom's level**: จำ, เข้าใจ, นำไปใช้, วิเคราะห์, ประเมิน, สร้างสรรค์
- **สามารถมีหลายระดับ Bloom's**: `<bloom_levels><level>เข้าใจ</level><level>นำไปใช้</level></bloom_levels>`

### ระดับความยาก:
- **ง่าย**: โจทย์พื้นฐาน 1-2 ขั้นตอน (คะแนน 1-2)
- **ปานกลาง**: โจทย์ที่ต้องใช้ความคิด 3-4 ขั้นตอน (คะแนน 3-4)
- **ยาก**: โจทย์ซับซ้อน ต้องวิเคราะห์เชิงลึก 5+ ขั้นตอน (คะแนน 5+)

### การให้คะแนน:
- Score ควรสะท้อนระดับความยากและเวลาที่ใช้ในการแก้ปัญหา
- โจทย์ที่ต้องใช้ Bloom's level สูงควรได้คะแนนมากกว่า

คุณพร้อมที่จะสร้างโจทย์คณิตศาสตร์คุณภาพสูงตามที่ได้รับมอบหมายแล้วหรือไม่?"""

USER_PROMPT = """
จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: พีชคณิต
2. สำหรับนักเรียน: ม.4
3. รูปแบบ: multiple_choice
4. ความยาก: ยาก
5. bloom level: เข้าใจ, นำไปใช้
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด
"""

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
/var/folders/gb/mhgb8669351fvrj7g23ydrdh0000gq/T/ipykernel_34453/1475852508.py:1: SyntaxWarning: invalid escape sequence '\p'
  SYSTEM_PROMPT = """


### Openrouter
- ChatGPT-5
- Gemini 2.5 Pro

In [32]:
BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = ["multiple_choice"]

N_JOBS = 8

In [ ]:
# OpenRouter and optional BYO provider keys from environment
try:
    # Load variables from a local .env if present
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")  # required
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # optional; required for OpenAI-routed models

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
# You can override the model via env var OPENROUTER_MODEL. Default aims at an OpenAI model; requires OPENAI_API_KEY.
MODEL_NAME = "openai/gpt-5"

if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY environment variable.")

In [34]:
bloom_combinations = [[b] for b in BLOOM_LEVELS] + [list(pair) for pair in combinations(BLOOM_LEVELS, 2)]
all_combinations = list(product(TOPICS, GRADE_LEVELS, QUESTION_TYPES, LEVELS, bloom_combinations))
print(f"Total combinations: {len(all_combinations)}")

Total combinations: 189


In [35]:
def create_user_prompt(topic, grade_level, question_type, difficulty, bloom_levels, num_problems=1):
    bloom_str = ", ".join(bloom_levels)
    return f"""จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: {topic}
2. สำหรับนักเรียน: {grade_level}
3. รูปแบบ: {question_type}
4. ความยาก: {difficulty}
5. bloom level: {bloom_str}
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด"""

In [40]:
def call_openrouter_api(user_prompt: str, system_prompt: str, model: str = MODEL_NAME):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.6,
        "top_p": 0.95,
        "top_k": 20,
        "max_tokens": 8000,
    }

    try:
        response = requests.post(f"{OPENROUTER_BASE_URL}/chat/completions", headers=headers, json=payload, timeout=120)

        print(response)
        print("-"*50)
        print(response.content)
        print("-"*50)
        print(response.text)

        if response.status_code == 200:
            response_json = response.json()
            message = response_json.get("choices", [{}])[0].get("message", {})
            content = message.get("content", "")
            reasoning = message.get("reasoning", None)
            return {
                "success": True,
                "error": "",
                "id": response_json.get("id"),
                "provider": response_json.get("provider"),
                "model": response_json.get("model") or model,
                "content": content,
                "reasoning": reasoning,
            }
        else:
            # Try to parse provider error body
            err_text = response.text
            try:
                err_json = response.json()
                err_text = json.dumps(err_json, ensure_ascii=False)
            except Exception:
                pass
            return {
                "success": False,
                "error": f"HTTP {response.status_code}: {err_text}",
                "content": "",
            }
    except requests.RequestException as e:
        return {
            "success": False,
            "error": f"Request error: {e}",
            "content": "",
        }

In [41]:
def generate_problem(params: Tuple[str, str, str, str, List[str], int]) -> Dict[str, Any]:
    topic, grade, q_type, difficulty, bloom = params

    time.sleep(random.uniform(0.5, 1.5))

    user_prompt = create_user_prompt(
        topic=topic,
        grade_level=grade,
        question_type=q_type,
        difficulty=difficulty,
        bloom_levels=bloom
    )

    api_result = call_openrouter_api(user_prompt, SYSTEM_PROMPT)

    result = {
        "topic": topic,
        "grade": grade,
        "question_type": q_type,
        "difficulty": difficulty,
        "bloom": bloom,
        "content": api_result.get("content", ""),
        "reasoning": api_result.get("reasoning"),
        "id": api_result.get("id"),
        "model": api_result.get("model"),
        "provider": api_result.get("provider"),
    }

    return result

In [42]:
def extract_think_and_content(output_str):
    think, content = "", ""
    if "<think>" in output_str and "</think>" in output_str:
        think = output_str.split("<think>")[1].split("</think>")[0].replace("\n", " ").strip()
    # Keep everything from <questions> ... to the end (single block) and remove newlines
    if "<questions>" in output_str:
        content = output_str.split("<questions>", 1)[1]
        content = "<questions>" + content
        content = content.replace("\n", " ").strip()
    return think, content

def save_result(think, content, run_id, params, result_meta=None, outdir="results"):
    """Save per-sample JSON file with think/content and metadata."""
    os.makedirs(outdir, exist_ok=True)

    record = {
        "id": run_id,
        "topic": params[0],
        "grade": params[1],
        "qtype": params[2],
        "level": params[3],
        "bloom": params[4],
        "think": think,
        "content": content,
        "meta": result_meta or {}
    }
    with open(f"{outdir}/result_{run_id}.json", "w", encoding="utf-8") as f:
        json.dump(record, f, ensure_ascii=False)

In [43]:
def process_one_combination(params, outdir="results"):
    topic, grade, qtype, level, bloom = params
    user_prompt = create_user_prompt(topic, grade, qtype, level, bloom)
    api = call_openrouter_api(user_prompt, SYSTEM_PROMPT)
    if not api:
        return None

    content = api.get("content", "")
    think, cleaned = extract_think_and_content(content)

    meta = {
        "provider": api.get("provider"),
        "model": api.get("model"),
    }
    run_id = api.get("id") or ""
    save_result(think=api.get("reasoning"), content=cleaned, run_id=run_id, params=params, result_meta=meta, outdir=outdir)
    return {
        "id": run_id,
        "topic": topic,
        "grade": grade,
        "qtype": qtype,
        "level": level,
        "bloom": bloom,
        "think": api.get("reasoning"),
        "content": cleaned,
    }

In [46]:
# Full system test 1 round
process_one_combination(all_combinations[0], outdir="test_results")

<Response [400]>
--------------------------------------------------
b'{"error":{"message":"Provider returned error","code":400,"metadata":{"raw":"{\\n  \\"error\\": {\\n    \\"message\\": \\"Your organization must be verified to stream this model. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.\\",\\n    \\"type\\": \\"invalid_request_error\\",\\n    \\"param\\": \\"stream\\",\\n    \\"code\\": \\"unsupported_value\\"\\n  }\\n}","provider_name":"OpenAI"}},"user_id":"user_2wUQtGsrEYfWRssFrND26wpf3XT"}'
--------------------------------------------------
{"error":{"message":"Provider returned error","code":400,"metadata":{"raw":"{\n  \"error\": {\n    \"message\": \"Your organization must be verified to stream this model. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can ta

{'id': '',
 'topic': 'พีชคณิต',
 'grade': 'ม.4',
 'qtype': 'multiple_choice',
 'level': 'ง่าย',
 'bloom': ['จำ'],
 'think': None,
 'content': ''}

In [ ]:
results = []

In [241]:
print(f"Generating {len(all_combinations)} problems using {N_JOBS} parallel jobs...")

for res in tqdm(
    Parallel(n_jobs=N_JOBS, return_as="generator")(
        delayed(process_one_combination)(params) for params in all_combinations
    ),
    total=len(all_combinations),
    desc="Generating"
):
    results.append(res)    

print("Done!")

Generating 189 problems using 8 parallel jobs...


Generating: 100%|██████████| 189/189 [14:16<00:00,  4.53s/it]

Done!


In [242]:
# Save
results = [r for r in results if r]
df = pd.DataFrame(results)
df.to_csv("all_results.csv", index=False, encoding="utf-8-sig")
df.to_parquet("all_results.parquet", index=False)

In [245]:
# Filter
df[df["id"].str.startswith("gen-")]

,id,topic,grade,qtype,level,bloom,think,content,reasoning
156,gen-1754968228-3ONu94irCnGA32wONH2y,พีชคณิต,ม.4,multiple_choice,ง่าย,[จำ],1. คิดโจทย์: พิจารณาโจทย์เกี่ยวกับพีชคณิตที่เก...,<questions> <question> <text>จงหาค่า $ x...,NaN
157,gen-1754968228-d5NlSrcr8P1554VqDHvI,พีชคณิต,ม.4,multiple_choice,ง่าย,[เข้าใจ],1. **คิดโจทย์เป็นภาษาอังกฤษ**: Create a multip...,<questions> <question> <text>จงหาค่า $ x...,NaN
158,gen-1754968228-f0iZIgURtIkUkkpLAZlr,พีชคณิต,ม.4,multiple_choice,ง่าย,[นำไปใช้],โจทย์ที่ต้องการสร้างคือโจทย์พีชคณิตสำหรับนักเร...,<questions> <question> <text>จงหาค่า $ x...,NaN
159,gen-1754968228-yqcqp0RmqwuGepnESVzg,พีชคณิต,ม.4,multiple_choice,ง่าย,[วิเคราะห์],1. **คิดโจทย์เป็นภาษาอังกฤษก่อน**: I need to c...,<questions> <question> <text>จงหาค่าที่ท...,NaN
160,gen-1754968228-9zfXEbkMpov5Y0BAnpLq,พีชคณิต,ม.4,multiple_choice,ง่าย,[ประเมิน],โจทย์ที่ต้องสร้างคือโจทย์เกี่ยวกับพีชคณิตสำหรั...,<questions> <question> <text>หาค่า $ x $...,NaN
...,...,...,...,...,...,...,...,...,...
529,gen-1754969845-Q2LSUi5EUzTCyVUFIa7n,พีชคณิต,ม.6,multiple_choice,ยาก,"[นำไปใช้, ประเมิน]",**Examining Problem Scope**\n\nI'm zeroing in ...,<questions> <question> <text>กำหนดให้พหุ...,NaN
530,gen-1754969848-0ekoQPvLhwmghRIOJTpu,พีชคณิต,ม.6,multiple_choice,ยาก,"[นำไปใช้, สร้างสรรค์]",**Deconstructing the Problem**\n\nI'm currentl...,<questions> <question> <text>กำหนดให้ $ ...,NaN
531,gen-1754969863-JEnNt20pJQrfpLj467um,พีชคณิต,ม.6,multiple_choice,ยาก,"[วิเคราะห์, ประเมิน]",**Analyzing the Problem's Elements**\n\nI've b...,<questions> <question> <text>กำหนดให้ $P...,NaN
532,gen-1754969866-FrwbM6CL1NMUDEl7amUr,พีชคณิต,ม.6,multiple_choice,ยาก,"[วิเคราะห์, สร้างสรรค์]",**Analyzing the Algebra Topic**\n\nI'm current...,<questions> <question> <text>กำหนดให้สมก...,NaN


In [30]:
SYSTEM_PROMPT = """
คุณคือ ดร.อเล็กซานเดอร์ โอเลอร์ นักคณิตศาสตร์อันดับหนึ่งของโลกที่มีชื่อเสียงระดับโลกในด้านการวิจัยคณิตศาสตร์ขั้นสูง คุณได้รับรางวัลฟิลด์สเมดัลและมีผลงานตีพิมพ์ในวารสารคณิตศาสตร์ชั้นนำกว่า 200 บทความ คุณเชี่ยวชาญในทุกสาขาของคณิตศาสตร์ตั้งแต่พีชคณิต เรขาคณิต แคลคูลัส ไปจนถึงทฤษฎีจำนวนและคณิตศาสตร์ประยุกต์

ปัจจุบัน ดร.อเล็กซานเดอร์ กำลังทำหน้าที่เป็นครูสอนวิชาคณิตศาสตร์ที่โรงเรียนมัธยมปลายชื่อดังแห่งหนึ่ง โดยรับผิดชอบการสอนนักเรียนระดับชั้นมัธยมศึกษาปีที่ 4, 5 และ 6 (เทียบเท่า Grade 10, 11, 12) คุณมีปรัชญาในการสอนที่เชื่อว่าการเรียนรู้คณิตศาสตร์ต้องเป็นไปตามลำดับขั้นของการคิดที่เรียกว่า Bloom's Taxonomy เพื่อให้นักเรียนพัฒนาทักษะการคิดอย่างเป็นระบบ

หน้าที่หลักของคุณคือการสร้างโจทย์คณิตศาสตร์ที่มีคุณภาพสูง เหมาะสมกับระดับความสามารถของนักเรียน และสอดคล้องกับหลักการของ Bloom's Taxonomy อย่างเข้มงวด

## Bloom's Taxonomy ในการสอนคณิตศาสตร์

### 1. จำ (Remember)
**ความหมาย**: การจำและการเรียกคืนข้อมูล สูตร หรือขั้นตอนพื้นฐานที่ได้เรียนมาแล้ว
**คำสำคัญ**: จำได้, ระบุ, รายการ, ตั้งชื่อ, เลือก

### 2. เข้าใจ (Understand)
**ความหมาย**: การเข้าใจความหมาย แนวคิด และสามารถอธิบายด้วยคำพูดของตนเองได้
**คำสำคัญ**: อธิบาย, แปล, สรุป, เปรียบเทียบ, แสดงให้เห็น

### 3. นำไปใช้ (Apply)
**ความหมาย**: การนำความรู้ สูตร หรือขั้นตอนที่เรียนมาไปใช้แก้ปัญหาในสถานการณ์ใหม่
**คำสำคัญ**: คำนวณ, แก้, ใช้, แสดง, ดำเนินการ

### 4. วิเคราะห์ (Analyze)
**ความหมาย**: การแยกแยะองค์ประกอบ วิเคราะห์ความสัมพันธ์ และเข้าใจโครงสร้างของปัญหา
**คำสำคัญ**: แยกแยะ, เปรียบเทียบ, ตรวจสอบ, ทดสอบ, วิเคราะห์

### 5. ประเมิน (Evaluate)
**ความหมาย**: การตัดสิน ประเมินค่า ให้เหตุผล และแสดงความคิดเห็นโดยใช้เกณฑ์ที่กำหนด
**คำสำคัญ**: ตัดสิน, ประเมิน, วิจารณ์, แสดงความคิดเห็น, ให้เหตุผล

### 6. สร้างสรรค์ (Create)
**ความหมาย**: การสร้างสรรค์สิ่งใหม่ ออกแบบ วางแผน หรือสร้างโจทย์ปัญหาขึ้นมาเอง
**คำสำคัญ**: สร้าง, ออกแบบ, วางแผน, เสนอ, พัฒนา

## ตัวอย่างโจทย์ตาม Bloom's Taxonomy

### จำ (Remember)
โจทย์: สูตรหาพื้นที่วงกลมคือข้อใด
A) $ A = \pi r^2 $  B) $ A = 2\pi r $  C) $ A = \pi d $  D) $ A = \frac{1}{2}\pi r^2 $

### เข้าใจ (Understand)
โจทย์: อธิบายความหมายของ $ \frac{dy}{dx} = 3x^2 $ ในทางเรขาคณิต

### นำไปใช้ (Apply)
โจทย์: จงหาค่า $ x $ จากสมการ $ 2x + 5 = 13 $

### วิเคราะห์ (Analyze)
โจทย์: เปรียบเทียบพฤติกรรมของฟังก์ชัน $ f(x) = x^2 $ และ $ g(x) = x^3 $ เมื่อ $ x > 1 $

### ประเมิน (Evaluate)
โจทย์: นักเรียนคนหนึ่งอ้างว่า $ \sqrt{a + b} = \sqrt{a} + \sqrt{b} $ เสมอ จงประเมินว่าข้อความนี้ถูกหรือผิด พร้อมให้เหตุผล

### สร้างสรรค์ (Create)
โจทย์: ออกแบบฟังก์ชันกำลังสองที่มีจุดยอดอยู่ที่ $ (2, -3) $ และผ่านจุด $ (0, 1) $

## ตัวอย่างโจทย์แบบผสม 5 โจทย์

### โจทย์ที่ 1 (เข้าใจ + นำไปใช้)
หากฟังก์ชัน $ f(x) = 2x - 3 $ จงหาค่า $ f(5) $ และอธิบายความหมายของผลลัพธ์

### โจทย์ที่ 2 (วิเคราะห์ + ประเมิน)
เปรียบเทียบวิธีแก้สมการ $ x^2 - 5x + 6 = 0 $ ด้วยการแยกตัวประกอบและสูตรกำลังสอง แล้วประเมินว่าวิธีใดมีประสิทธิภาพมากกว่า

### โจทย์ที่ 3 (นำไปใช้ + วิเคราะห์)
ร้านค้าแห่งหนึ่งขายสินค้าในราคา $ 100x - x^2 $ บาท เมื่อขาย $ x $ ชิ้น จงหาจำนวนสินค้าที่ขายได้เงินสูงสุด

### โจทย์ที่ 4 (เข้าใจ + สร้างสรรค์)
จากกราฟ $ y = \sin x $ จงสร้างฟังก์ชันใหม่ที่มีแอมพลิจูดเป็น 3 เท่า และอธิบายการเปลี่ยนแปลง

### โจทย์ที่ 5 (วิเคราะห์ + ประเมิน + สร้างสรรค์)
นักเรียนสองคนแก้โจทย์หาค่าสูงสุดของ $ f(x) = -x^2 + 4x + 1 $ ได้คำตอบต่างกัน คนหนึ่งได้ $ x = 2 $ อีกคนได้ $ x = -2 $ จงวิเคราะห์ว่าใครถูก ประเมินข้อผิดพลาด และสร้างวิธีตรวจสอบคำตอบ

---

## คำสั่งการทำงาน

เมื่อได้รับโจทย์ให้สร้างข้อสอบคณิตศาสตร์ ให้ทำตามขั้นตอนดังนี้:

### ขั้นตอนที่ 1: การคิดและวางแผน
ในแท็ก `<think></think>` ให้ทำการ:
1. **คิดโจทย์เป็นภาษาอังกฤษก่อน** - เพื่อให้ได้แนวคิดที่ชัดเจน
2. **ทดลองหาคำตอบ** - ตรวจสอบว่าโจทย์สามารถแก้ได้จริง
3. **หากแก้ไม่ได้ ให้เปลี่ยนโจทย์** - แล้วทดลองใหม่จนได้โจทย์ที่ดี
4. **เมื่อได้โจทย์ที่สมบูรณ์แล้ว** - สร้างตัวเลือกคำตอบสำหรับ multiple choice (หากต้องการ)
5. **ตรวจสอบความสอดคล้องกับ Bloom's taxonomy** ที่กำหนด

### ขั้นตอนที่ 2: การเขียนผลลัพธ์
หลังจาก `<think></think>` แล้ว ให้เขียนโจทย์ในรูปแบบ XML ตามตัวอย่างนี้:

```xml
<questions>
  <question>
    <text>โจทย์คณิตศาสตร์ที่มี KaTeX formatting เช่น $ 4x + 3 = 2x + 9 $</text>
    <type>multiple_choice</type>
    <options>
      <option>$ ตัวเลือก1 $</option>
      <option>$ ตัวเลือก2 $</option>
      <option>$ ตัวเลือก3 $</option>
      <option>$ ตัวเลือก4 $</option>
    </options>
    <correct_answer>$ คำตอบที่ถูก $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: อธิบายด้วย KaTeX เช่น $ 4x - 2x + 3 = 9 $
      ขั้นตอนที่ 2: $ 2x + 3 = 9 $
      ขั้นตอนที่ 3: $ 2x = 6 $
      ขั้นตอนที่ 4: $ x = 3 $
    </explanation>
    <score>2</score>
    <difficulty>ง่าย</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
      <level>วิเคราะห์</level>
    </bloom_levels>
  </question>

  <question>
    <text>จงหาค่า $ x $ ที่ทำให้ $ 3x - 7 = 2x + 5 $</text>
    <type>short_answer</type>
    <correct_answer>$ x = 12 $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: นำ $ 2x $ ไปยังข้างซ้าย และ $ 7 $ ไปยังข้างขวา
      $ 3x - 2x = 5 + 7 $
      ขั้นตอนที่ 2: คำนวณ
      $ x = 12 $
    </explanation>
    <score>2</score>
    <difficulty>ปานกลาง</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
    </bloom_levels>
  </question>
</questions>
```

## หมายเหตุสำคัญ

### รูปแบบการเขียน:
- **ใช้ KaTeX format** `$ ... $` สำหรับตัวเลข สูตร และนิพจน์คณิตศาสตร์ทั้งหมด
- **ใช้คำไทยสำหรับ Bloom's level**: จำ, เข้าใจ, นำไปใช้, วิเคราะห์, ประเมิน, สร้างสรรค์
- **สามารถมีหลายระดับ Bloom's**: `<bloom_levels><level>เข้าใจ</level><level>นำไปใช้</level></bloom_levels>`

### ระดับความยาก:
- **ง่าย**: โจทย์พื้นฐาน 1-2 ขั้นตอน (คะแนน 1-2)
- **ปานกลาง**: โจทย์ที่ต้องใช้ความคิด 3-4 ขั้นตอน (คะแนน 3-4)
- **ยาก**: โจทย์ซับซ้อน ต้องวิเคราะห์เชิงลึก 5+ ขั้นตอน (คะแนน 5+)

### การให้คะแนน:
- Score ควรสะท้อนระดับความยากและเวลาที่ใช้ในการแก้ปัญหา
- โจทย์ที่ต้องใช้ Bloom's level สูงควรได้คะแนนมากกว่า

คุณพร้อมที่จะสร้างโจทย์คณิตศาสตร์คุณภาพสูงตามที่ได้รับมอบหมายแล้วหรือไม่?"""

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
/var/folders/gb/mhgb8669351fvrj7g23ydrdh0000gq/T/ipykernel_61123/2134333895.py:1: SyntaxWarning: invalid escape sequence '\p'
  SYSTEM_PROMPT = """
